# 笨办法融合一下

# lgb_mae*0.3+lgb_mse*0.3 +cat*0.3 +xgb*0.1)*0.5 +(guize1*0.5+guize2*0.5)*0.5
#LGB 0.636 mse
#0.639 mae

# 0.50151896000 lgb

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.6-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.6-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [54]:
!mkdir -p GuodongDrive
!google-drive-ocamlfuse GuodongDrive
import os
os.chdir("GuodongDrive")
os.chdir("ccf/carsale")
!ls

0825ccf_carsale.csv	   baseline2.ipynb
0825ccf_car_sales_lgb.csv  catboost_info
083101_car_sale.csv	   ccf_car_sales082601.csv
083101sub.csv		   ccf_car_sales082602.csv
083103_car_sale.csv	   ccf_car_sales.csv
083103sub.csv		   ccf_car_sales_lgb.csv
090101car_sales.csv	   evaluation_public.csv
090102car_sales.csv	   evaluation_public_new.csv
090103car_sales.csv	   evaluation_public_xishao.csv
090201car_sales.csv	   guize1-new.ipynb
baseline082601.ipynb	   GuodongDrive
baseline082701.ipynb	   ronghe_car_sales.csv
baseline082702.ipynb	   submit_example.csv
baseline082703.ipynb	   train_sales_data.csv
baseline082704.ipynb	   train_sales_new.csv
baseline083101.ipynb	   train_search_data.csv
baseline083102.ipynb	   train_user_reply_data.csv
baseline090101.ipynb	   Untitled0.ipynb
baseline1_copy.ipynb	   xishao_car_sale.csv
baseline1.ipynb		   统计_car_sales.csv


In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import LabelEncoder

import xgboost as xgb
import lightgbm as lgb
import catboost as ctb

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline 
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

In [2]:
from sklearn import linear_model, svm, gaussian_process
from sklearn.ensemble import RandomForestRegressor

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
def score(data, pred='pred_label', label='label', group='model'):
  
    data['pred_label'] = data['pred_label'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
    data_agg = data.groupby('model').agg({
        pred: list,
        label: [list, 'mean'],

    }).reset_index()

    data_agg.columns = ['_'.join(col).strip() for col in data_agg.columns]
    nrmse_score = []
    for raw in data_agg[['{0}_list'.format(pred), '{0}_list'.format(label), '{0}_mean'.format(label)]].values:
        nrmse_score.append(
            mse(raw[0], raw[1]) ** 0.5 / raw[2]
        )
    print(1 - np.mean(nrmse_score))
    return 1 - np.mean(nrmse_score)

思路：  
- 这种popularity	carCommentVolum	newsReplyVolum我感觉都是增长的，可以预测出来,我们可以单独用模型将接下来四个月份的这三个指标预测出来

- 在统计分数的时候所用的验证集，已经作为模型的验证集训练过，所以得到的分数会偏高
应该用split先将train_set分为train和valid，再去用17年最后的四个月作为统计分数的指标

- 可以用KFold进行train_set和valid_set的训练

- 他在训练时将特征model和type换成另一种表现方式，不妨把这种表现方式设为新的特征，model和type依然保留作为特征

-预测流量指数考虑clfs = {
        'svm':svm.SVR(), 
        'RandomForestRegressor':RandomForestRegressor(n_estimators=400),
        'BayesianRidge':linear_model.BayesianRidge()
       }
for clf in clfs:
    try:
        clfs[clf].fit(X_train, y_train)
        y_pred = clfs[clf].predict(X_test)
        print(clf + " cost:" + str(np.sum(y_pred-y_test)/len(y_pred)) )

In [5]:
train_sales_data = pd.read_csv('train_sales_data.csv')
train_search_data = pd.read_csv('train_search_data.csv')
train_user_reply_data = pd.read_csv('train_user_reply_data.csv')
test = pd.read_csv('evaluation_public.csv')

In [6]:
train_sales_data['bodyType'].unique()

array(['SUV', 'Sedan', 'MPV', 'Hatchback'], dtype=object)

In [0]:
test.tail(10)

,id,province,adcode,model,regYear,regMonth,forecastVolum,qushi_v
5270,5359,河北,130000,a9a43d1a7ecbe75d,2018,4,NaN,4.009867
5271,5360,河南,410000,a9a43d1a7ecbe75d,2018,4,NaN,3.543448
5272,5361,浙江,330000,a9a43d1a7ecbe75d,2018,4,NaN,11.047862
5273,5362,湖北,420000,a9a43d1a7ecbe75d,2018,4,NaN,6.149299
5274,5363,湖南,430000,a9a43d1a7ecbe75d,2018,4,NaN,9.926640
5275,5364,福建,350000,a9a43d1a7ecbe75d,2018,4,NaN,3.347938
5276,5365,辽宁,210000,a9a43d1a7ecbe75d,2018,4,NaN,2.723840
5277,5366,重庆,500000,a9a43d1a7ecbe75d,2018,4,NaN,3.632789
5278,5367,陕西,610000,a9a43d1a7ecbe75d,2018,4,NaN,7.321233
5279,5368,黑龙江,230000,a9a43d1a7ecbe75d,2018,4,NaN,1.811766


In [7]:
data = pd.concat([train_sales_data, test], ignore_index=True)
data = data.merge(train_search_data, 'left', on=['province', 'adcode', 'model', 'regYear', 'regMonth'])
data = data.merge(train_user_reply_data, 'left', on=['model', 'regYear', 'regMonth'])
data.head(10)

,adcode,bodyType,forecastVolum,id,model,province,regMonth,regYear,salesVolume,popularity,carCommentVolum,newsReplyVolum
0,310000,SUV,NaN,NaN,3c974920a76ac9c1,上海,1,2016,292.0,1479.0,11.0,106.0
1,530000,SUV,NaN,NaN,3c974920a76ac9c1,云南,1,2016,466.0,1594.0,11.0,106.0
2,150000,SUV,NaN,NaN,3c974920a76ac9c1,内蒙古,1,2016,257.0,1479.0,11.0,106.0
3,110000,SUV,NaN,NaN,3c974920a76ac9c1,北京,1,2016,408.0,2370.0,11.0,106.0
4,510000,SUV,NaN,NaN,3c974920a76ac9c1,四川,1,2016,610.0,3562.0,11.0,106.0
5,340000,SUV,NaN,NaN,3c974920a76ac9c1,安徽,1,2016,206.0,1314.0,11.0,106.0
6,370000,SUV,NaN,NaN,3c974920a76ac9c1,山东,1,2016,503.0,3476.0,11.0,106.0
7,140000,SUV,NaN,NaN,3c974920a76ac9c1,山西,1,2016,236.0,1422.0,11.0,106.0
8,440000,SUV,NaN,NaN,3c974920a76ac9c1,广东,1,2016,3635.0,7182.0,11.0,106.0
9,450000,SUV,NaN,NaN,3c974920a76ac9c1,广西,1,2016,450.0,1163.0,11.0,106.0


In [8]:
data['label'] = data['salesVolume']
data['id'] = data['id'].fillna(0).astype(int)
del data['salesVolume'], data['forecastVolum']
data['bodyType_2'] = data['model'].map(train_sales_data.drop_duplicates('model').set_index('model')['bodyType'])
data['model'].head()
data['model_2'] = data['model']
#一种model可能对应多种type

In [9]:
for i in ['bodyType_2', 'model_2']:
    data[i] = data[i].map(dict(zip(data[i].unique(), range(data[i].nunique()))))#用这个函数可以查看数据有多少个不同值。

data.head(10)

,adcode,bodyType,id,model,province,regMonth,regYear,popularity,carCommentVolum,newsReplyVolum,label,bodyType_2,model_2
0,310000,SUV,0,3c974920a76ac9c1,上海,1,2016,1479.0,11.0,106.0,292.0,0,0
1,530000,SUV,0,3c974920a76ac9c1,云南,1,2016,1594.0,11.0,106.0,466.0,0,0
2,150000,SUV,0,3c974920a76ac9c1,内蒙古,1,2016,1479.0,11.0,106.0,257.0,0,0
3,110000,SUV,0,3c974920a76ac9c1,北京,1,2016,2370.0,11.0,106.0,408.0,0,0
4,510000,SUV,0,3c974920a76ac9c1,四川,1,2016,3562.0,11.0,106.0,610.0,0,0
5,340000,SUV,0,3c974920a76ac9c1,安徽,1,2016,1314.0,11.0,106.0,206.0,0,0
6,370000,SUV,0,3c974920a76ac9c1,山东,1,2016,3476.0,11.0,106.0,503.0,0,0
7,140000,SUV,0,3c974920a76ac9c1,山西,1,2016,1422.0,11.0,106.0,236.0,0,0
8,440000,SUV,0,3c974920a76ac9c1,广东,1,2016,7182.0,11.0,106.0,3635.0,0,0
9,450000,SUV,0,3c974920a76ac9c1,广西,1,2016,1163.0,11.0,106.0,450.0,0,0


In [10]:
data['mt'] = (data['regYear'] - 2016) * 12 + data['regMonth']

# 'popularity', 'carCommentVolum', 'newsReplyVolum','label'/

shift_feat = []

data['model_adcode'] = data['adcode'] + data['model_2']
data['model_adcode_mt'] = data['model_adcode'] * 100 + data['mt']
for i in [11]:
    i = i + 1
    shift_feat.append('shift_model_adcode_mt_label_{0}'.format(i))
    data['model_adcode_mt_{0}'.format(i)] = data['model_adcode_mt'] + i
    data_last = data[~data.label.isnull()].set_index('model_adcode_mt_{0}'.format(i))
    data['shift_model_adcode_mt_label_{0}'.format(i)] = data['model_adcode_mt'].map(data_last['label'])

shift_feat

['shift_model_adcode_mt_label_12']

In [11]:
num_feat = ['adcode', 'regMonth', 'regYear', 'popularity', 'carCommentVolum', 'newsReplyVolum', 'bodyType_2', 'model_2']+ shift_feat
cate_feat = ['bodyType', 'model', 'province']

# lbl = LabelEncoder()
# for i in cate_feat:
#   data[i] = lbl.fit_transform(data[i].astype(str))
for i in cate_feat:
    data[i] = data[i].astype('category')
features = num_feat + cate_feat

**划分训练集，线下验证集，测试集index**

In [12]:
# data['n_label'] = data['label'] / data.groupby('model')['label'].transform('mean')
train_idx = (data['mt'] <= 20)

valid_idx = (data['mt'].between(21, 24))

test_idx = (data['mt'] > 24)

In [13]:
data['model_weight'] = data.groupby('model_2')['label'].transform('mean')
data['n_label'] = data['label'] / data['model_weight']

In [14]:
train_x = data[train_idx][features]
train_y = data[train_idx]['n_label']
print("train_x is\n")
#print(train_x)
print("-----------------------------------")
valid_x = data[valid_idx][features]
valid_y = data[valid_idx]['n_label']
print("valid_x is\n")
#print(valid_x)
print("-----------------------------------")

train_x is

-----------------------------------
valid_x is

-----------------------------------


In [14]:
ctb_params = {
    'n_estimators': 6000,
    'learning_rate': 0.02,
    'random_seed': 2019,
    'reg_lambda': 0.08,
    'subsample': 0.7,
    'bootstrap_type': 'Bernoulli',
    'boosting_type': 'Plain',
    'one_hot_max_size': 10,
    'rsm': 0.5,
    'leaf_estimation_iterations': 5,
    'use_best_model': True,
    'max_depth': 6,
    'verbose': -1,
    'thread_count': 4
}


ctb_model = ctb.CatBoostRegressor(**ctb_params, cat_features=cate_feat, early_stopping_rounds=100)

ctb_model.fit(train_x, train_y,
           eval_set=[(valid_x, valid_y)],
                      #eval_metric='mae',
                     
                      #sample_weight=train_data.loc[train_idx]['sample_weight'],
                      verbose=100)

0:	learn: 1.3017983	test: 1.5183241	best: 1.5183241 (0)	total: 136ms	remaining: 13m 38s
100:	learn: 0.6351388	test: 0.8162979	best: 0.8162979 (100)	total: 3.2s	remaining: 3m 6s
200:	learn: 0.5237058	test: 0.6404836	best: 0.6404836 (200)	total: 6.25s	remaining: 3m
300:	learn: 0.4815920	test: 0.5857065	best: 0.5857065 (300)	total: 9.26s	remaining: 2m 55s
400:	learn: 0.4530496	test: 0.5586910	best: 0.5586910 (400)	total: 12.5s	remaining: 2m 54s
500:	learn: 0.4321139	test: 0.5422140	best: 0.5422140 (500)	total: 15.5s	remaining: 2m 49s
600:	learn: 0.4133561	test: 0.5281193	best: 0.5281035 (599)	total: 18.5s	remaining: 2m 45s
700:	learn: 0.3966433	test: 0.5185434	best: 0.5185434 (700)	total: 21.5s	remaining: 2m 42s
800:	learn: 0.3830909	test: 0.5124849	best: 0.5124849 (800)	total: 24.5s	remaining: 2m 39s
900:	learn: 0.3717200	test: 0.5076737	best: 0.5076120 (898)	total: 27.5s	remaining: 2m 35s
1000:	learn: 0.3616507	test: 0.5032858	best: 0.5032858 (1000)	total: 30.7s	remaining: 2m 33s
1100:	

In [15]:
data['pred_label'] = ctb_model.predict(data[features].fillna(0)) * data['model_weight']
data['ctb'] =data['pred_label'] 

In [16]:
best_score = score(data[valid_idx])

0.6202980332493366


In [17]:
ctb_model.n_estimators = 10000

ctb_model.fit(data[~test_idx][features], data[~test_idx]['n_label'], eval_set=[(valid_x, valid_y)],cat_features=cate_feat,verbose=100)

data['forecastVolum'] = ctb_model.predict(data[features].fillna(0))   * data['model_weight']
sub = data[test_idx][['id']]
sub['forecastVolum'] = data[test_idx]['forecastVolum'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
sub.head(10)
sub.to_csv('0902xtb.csv', index=False)

0:	learn: 1.3393961	test: 1.5147387	best: 1.5147387 (0)	total: 33.4ms	remaining: 3m 20s
100:	learn: 0.6408931	test: 0.6762768	best: 0.6762768 (100)	total: 3.4s	remaining: 3m 18s
200:	learn: 0.5282867	test: 0.5473581	best: 0.5473581 (200)	total: 6.74s	remaining: 3m 14s
300:	learn: 0.4857088	test: 0.4991373	best: 0.4991373 (300)	total: 9.93s	remaining: 3m 7s
400:	learn: 0.4573631	test: 0.4715907	best: 0.4715907 (400)	total: 13.1s	remaining: 3m 2s
500:	learn: 0.4373318	test: 0.4514052	best: 0.4514052 (500)	total: 16.4s	remaining: 2m 59s
600:	learn: 0.4209861	test: 0.4353594	best: 0.4353594 (600)	total: 19.7s	remaining: 2m 56s
700:	learn: 0.4058238	test: 0.4197957	best: 0.4197957 (700)	total: 22.9s	remaining: 2m 53s
800:	learn: 0.3928628	test: 0.4053943	best: 0.4053943 (800)	total: 26.2s	remaining: 2m 49s
900:	learn: 0.3814894	test: 0.3929611	best: 0.3929611 (900)	total: 29.5s	remaining: 2m 46s
1000:	learn: 0.3714172	test: 0.3808815	best: 0.3808815 (1000)	total: 32.8s	remaining: 2m 43s
110

# lgb mse

In [18]:
num_feat = ['adcode', 'regMonth', 'regYear', 'popularity', 'carCommentVolum', 'newsReplyVolum', 'bodyType_2', 'model_2']+ shift_feat
cate_feat = ['bodyType', 'model', 'province']

# lbl = LabelEncoder()
# for i in cate_feat:
#   data[i] = lbl.fit_transform(data[i].astype(str))
for i in cate_feat:
    data[i] = data[i].astype('category')
features = num_feat + cate_feat

In [19]:
train_x = data[train_idx][features]
train_y = data[train_idx]['n_label']
print("train_x is\n")
#print(train_x)
print("-----------------------------------")
valid_x = data[valid_idx][features]
valid_y = data[valid_idx]['n_label']
print("valid_x is\n")
#print(valid_x)
print("-----------------------------------")

train_x is

-----------------------------------
valid_x is

-----------------------------------


In [15]:




lgb_model = lgb.LGBMRegressor(
    num_leaves=64, reg_alpha=1, reg_lambda=0.1, objective='mse',
    max_depth=-1, learning_rate=0.05, min_child_samples=20, random_state=np.random.randint(1000),
    n_estimators=6000, subsample=0.7, colsample_bytree=0.7,n_jobs=-1
)

lgb_model.fit(train_x, train_y, eval_set=[
    (valid_x, valid_y),
], categorical_feature=cate_feat, early_stopping_rounds=100, verbose=100)

#也就是说我们在训练集和验证集上训练完了之后再让它重新预测整个数据集，让其中的valid和vaild



Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l2: 0.245083
[200]	valid_0's l2: 0.217479
[300]	valid_0's l2: 0.211469
[400]	valid_0's l2: 0.208165
[500]	valid_0's l2: 0.205436
[600]	valid_0's l2: 0.203275
[700]	valid_0's l2: 0.20211
[800]	valid_0's l2: 0.201059
[900]	valid_0's l2: 0.200388
[1000]	valid_0's l2: 0.199784
[1100]	valid_0's l2: 0.199313
[1200]	valid_0's l2: 0.199069
[1300]	valid_0's l2: 0.1989
[1400]	valid_0's l2: 0.198668
[1500]	valid_0's l2: 0.198268
[1600]	valid_0's l2: 0.198138
[1700]	valid_0's l2: 0.197808
[1800]	valid_0's l2: 0.197555
[1900]	valid_0's l2: 0.197336
[2000]	valid_0's l2: 0.197301
[2100]	valid_0's l2: 0.197191
[2200]	valid_0's l2: 0.197033
Early stopping, best iteration is:
[2195]	valid_0's l2: 0.197027


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
              importance_type='split', learning_rate=0.05, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=6000, n_jobs=-1, num_leaves=64, objective='mse',
              random_state=698, reg_alpha=1, reg_lambda=0.1, silent=True,
              subsample=0.7, subsample_for_bin=200000, subsample_freq=0)

In [16]:
data['pred_label'] = lgb_model.predict(data[features]) * data['model_weight']


In [18]:
data['lgb_mse'] =data['pred_label'] 
best_score = score(data[valid_idx])




0.6338625159298399


In [19]:
lgb_model.n_estimators = 6000

lgb_model.fit(data[~test_idx][features], data[~test_idx]['n_label'], categorical_feature=cate_feat)

data['forecastVolum'] = lgb_model.predict(data[features])   * data['model_weight']
sub = data[test_idx][['id']]
sub['forecastVolum'] = data[test_idx]['forecastVolum'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
sub.head(10)
sub.to_csv('0905gb_mse.csv', index=False)

# lgb mae

In [20]:
lgb_model = lgb.LGBMRegressor(
    num_leaves=64, reg_alpha=1, reg_lambda=0.1, objective='mae',
    max_depth=-1, learning_rate=0.05, min_child_samples=20, random_state=np.random.randint(1000),
    n_estimators=6000, subsample=0.7, colsample_bytree=0.7,n_jobs=-1
)

lgb_model.fit(train_x, train_y, eval_set=[
    (valid_x, valid_y),
], categorical_feature=cate_feat, early_stopping_rounds=100, verbose=100)

#也就是说我们在训练集和验证集上训练完了之后再让它重新预测整个数据集，让其中的valid和vaild

Training until validation scores don't improve for 100 rounds.
[100]	valid_0's l1: 0.31382
[200]	valid_0's l1: 0.281314
[300]	valid_0's l1: 0.275241
[400]	valid_0's l1: 0.270725
[500]	valid_0's l1: 0.267879
[600]	valid_0's l1: 0.266247
[700]	valid_0's l1: 0.265128
[800]	valid_0's l1: 0.264404
[900]	valid_0's l1: 0.263551
[1000]	valid_0's l1: 0.262537
[1100]	valid_0's l1: 0.262034
[1200]	valid_0's l1: 0.261804
[1300]	valid_0's l1: 0.261676
[1400]	valid_0's l1: 0.261366
[1500]	valid_0's l1: 0.261273
Early stopping, best iteration is:
[1438]	valid_0's l1: 0.261244


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.7,
              importance_type='split', learning_rate=0.05, max_depth=-1,
              min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=6000, n_jobs=-1, num_leaves=64, objective='mae',
              random_state=518, reg_alpha=1, reg_lambda=0.1, silent=True,
              subsample=0.7, subsample_for_bin=200000, subsample_freq=0)

In [21]:
data['pred_label'] = lgb_model.predict(data[features]) * data['model_weight']
data['lgb_mae'] =data['pred_label'] 


In [22]:
best_score = score(data[valid_idx])

0.6294606071067068


In [24]:
lgb_model.n_estimators = 6000

lgb_model.fit(data[~test_idx][features], data[~test_idx]['n_label'], categorical_feature=cate_feat)

data['forecastVolum'] = lgb_model.predict(data[features])   * data['model_weight']
sub = data[test_idx][['id']]
sub['forecastVolum'] = data[test_idx]['forecastVolum'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
sub.head(10)
sub.to_csv('0905gb_mae.csv', index=False)

# xgb

In [28]:
num_feat = ['adcode', 'regMonth', 'regYear', 'popularity', 'carCommentVolum', 'newsReplyVolum', 'bodyType_2', 'model_2']+ shift_feat
cate_feat = ['bodyType', 'model', 'province']

lbl = LabelEncoder()
for i in cate_feat:
  data[i] = lbl.fit_transform(data[i].astype(str))
# for i in cate_feat:
#     data[i] = data[i].astype('category')
# features = num_feat + cate_feat

In [29]:
train_x = data[train_idx][features]
train_y = data[train_idx]['n_label']
print("train_x is\n")
#print(train_x)
print("-----------------------------------")
valid_x = data[valid_idx][features]
valid_y = data[valid_idx]['n_label']
print("valid_x is\n")
#print(valid_x)
print("-----------------------------------")

train_x is

-----------------------------------
valid_x is

-----------------------------------


In [30]:
xgb_model = xgb.XGBRegressor(
    max_depth=6,
    learning_rate=0.1,
    #objective='reg:squarederror',
    eval_metric='mae',
    subsample=0.8, colsample_bytree=0.5,
    n_estimators=6000,
    reg_alpha=1,
    reg_lambda=0.01,
    min_child_weight=20,
    n_jobs=-1,
   
)
xgb_model.fit(train_x, train_y, eval_set=[
    (valid_x, valid_y),
] ,early_stopping_rounds=100, verbose=100)

[0]	validation_0-mae:0.697464
Will train until validation_0-mae hasn't improved in 100 rounds.
[100]	validation_0-mae:0.347672
[200]	validation_0-mae:0.333725
[300]	validation_0-mae:0.324919
[400]	validation_0-mae:0.31852
[500]	validation_0-mae:0.316557
[600]	validation_0-mae:0.315082
[700]	validation_0-mae:0.312972
[800]	validation_0-mae:0.311815
[900]	validation_0-mae:0.310314
[1000]	validation_0-mae:0.309885
[1100]	validation_0-mae:0.30947
[1200]	validation_0-mae:0.30946
Stopping. Best iteration:
[1120]	validation_0-mae:0.309159



XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bytree=0.5, eval_metric='mae', gamma=0,
             learning_rate=0.1, max_delta_step=0, max_depth=6,
             min_child_weight=20, missing=None, n_estimators=6000, n_jobs=-1,
             nthread=None, objective='reg:linear', random_state=0, reg_alpha=1,
             reg_lambda=0.01, scale_pos_weight=1, seed=None, silent=True,
             subsample=0.8)

In [31]:
data['pred_label'] = xgb_model.predict(data[features].fillna(0)) * data['model_weight']
data['xgb'] =data['pred_label'] 

In [32]:
best_score = score(data[valid_idx])

0.5828131634631482


In [33]:
xgb_model.n_estimators = 5000

xgb_model.fit(data[~test_idx][features], data[~test_idx]['n_label'])

data['forecastVolum'] = xgb_model.predict(data[features])   * data['model_weight']
sub = data[test_idx][['id']]
sub['forecastVolum'] = data[test_idx]['forecastVolum'].apply(lambda x: 0 if x < 0 else x).round().astype(int)
sub.head(10)
sub.to_csv('0102xgb.csv', index=False)

In [52]:
# sub_ctb = pd.read_csv("0902xtb.csv").rename(columns={'forecastVolum':'forecastVolum1'})
# sub_ctb = pd.read_csv("0902lgb_mse.csv").rename(columns={'forecastVolum':'forecastVolum2'})
# sub_lgb_mae = pd.read_csv("0902lgb_mae.csv").rename(columns={'forecastVolum':'forecastVolum3'})
# sub_xgb = pd.read_csv("0102xgb.csv").rename(columns={'forecastVolum':'forecastVolum4'})

In [55]:
sub_ctb = pd.read_csv("0902xtb.csv")
sub_ctb = pd.read_csv("0902lgb_mse.csv")
sub_lgb_mae = pd.read_csv("0902lgb_mae.csv")
sub_xgb = pd.read_csv("0102xgb.csv")

In [50]:
# rule=rule2.merge(rule1,on=["adcode", "model", "province","regMonth"],how='left').rename(columns={'forecastVolum':'forecastVolum2'})
# result.head(10)

,id,forecastVolum
0,1,116
1,2,264
2,3,60
3,4,255
4,5,303
5,6,106
6,7,239
7,8,80
8,9,1814
9,10,331


In [56]:
sub_ctb.head()

sub_lgb_mae.head()
sub_xgb.head()

,id,forecastVolum
0,1,133
1,2,244
2,3,40
3,4,92
4,5,81


In [38]:
sub_lgb_mse.head()

,id,forecastVolum
0,1,206
1,2,285
2,3,116
3,4,262
4,5,273


In [57]:
result = sub_lgb_mse
result['forecastVolum'] = sub_ctb['forecastVolum']*0.4+sub_lgb_mae['forecastVolum']*0.4+sub_lgb_mse['forecastVolum']*0.2
result.head()

,id,forecastVolum
0,1,203.2
1,2,288.4
2,3,133.6
3,4,262.8
4,5,277.8


In [58]:
result['forecastVolum'] = result['forecastVolum'].round().astype(int)
result.head()

,id,forecastVolum
0,1,203
1,2,288
2,3,134
3,4,263
4,5,278


In [59]:
result2 = pd.read_csv("rongheguize_0901.csv")

result2.head(5)

,id,forecastVolum
0,1,289
1,2,378
2,3,154
3,4,344
4,5,402


In [45]:
result2.shape

(5280, 2)

In [60]:
result2['forecastVolum'] = result['forecastVolum']*0.5 + result2['forecastVolum']*0.5
result2.head()

,id,forecastVolum
0,1,246.0
1,2,333.0
2,3,144.0
3,4,303.5
4,5,340.0


In [61]:
result2['forecastVolum'] = result2['forecastVolum'].round().astype(int)
result2.head()

,id,forecastVolum
0,1,246
1,2,333
2,3,144
3,4,304
4,5,340


In [62]:
result2.to_csv('stacking1.csv', index=False)

In [0]:

# def get_predict_w(model, data, label='label', feature=[], cate_feature=[], random_state=2018, n_splits=5,
#                   model_type='lgb'):
#     if 'sample_weight' not in data.keys():
#         data['sample_weight'] = 1
#     model.random_state = random_state
#     predict_label = 'predict_' + label
#     kfold = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
#     data[predict_label] = 0
#     test_index = (data[label].isnull()) | (data[label] == -1)
#     train_data = data[~test_index].reset_index(drop=True)
#     test_data = data[test_index]
#
#     for train_idx, val_idx in kfold.split(train_data):
#         model.random_state = model.random_state + 1
#
#         train_x = train_data.loc[train_idx][feature]
#         train_y = train_data.loc[train_idx][label]
#
#         test_x = train_data.loc[val_idx][feature]
#         test_y = train_data.loc[val_idx][label]
#         if model_type == 'lgb':
#             try:
#                 model.fit(train_x, train_y, eval_set=[(test_x, test_y)], early_stopping_rounds=100,
#                           # eval_metric='mae',
#                           categorical_feature=cate_feature,
#                           sample_weight=train_data.loc[train_idx]['sample_weight'],
#                           verbose=100)
#             except:
#                 model.fit(train_x, train_y, eval_set=[(test_x, test_y)], early_stopping_rounds=100,
#                           eval_metric='mae',
#                           # categorical_feature=cate_feature,
#                           sample_weight=train_data.loc[train_idx]['sample_weight'],
#                           verbose=100)
#         elif model_type == 'ctb':
#             model.fit(train_x, train_y, eval_set=[(test_x, test_y)], early_stopping_rounds=100,
#                       # eval_metric='mae',
#                       cat_features=cate_feature,
#                       sample_weight=train_data.loc[train_idx]['sample_weight'],
#                       verbose=100)
#         train_data.loc[val_idx, predict_label] = model.predict(test_x)
#         if len(test_data) != 0:
#             test_data[predict_label] = test_data[predict_label] + model.predict(test_data[feature])
#     test_data[predict_label] = test_data[predict_label] / n_splits
#     print(mse(train_data[label], train_data[predict_label]), train_data[predict_label].mean(),
#           test_data[predict_label].mean())
#     return pd.concat([train_data, test_data], sort=True, ignore_index=True), predict_label
#
#
# data, predict_label = get_predict_w(lgb_model, data, label='label',
#                                     feature=features, cate_feature=cate_feat,
#                                     random_state=2019, n_splits=5)
#
# # 自定义评价函数
# model = data[valid_idx]['model'].values
#
#
# def n_rmse(preds, labels):
#     valid_data = pd.DataFrame({
#         'label': labels,
#         'pred_label': preds,
#         'model': model
#     })
#     n_rmse = score(valid_data)
#     return 'n_rmse', n_rmse, True
#
# #
# # data['lgb'] = data[predict_label]
# #
# # data['forecastVolum'] = data['lgb'].apply(lambda x: 0 if x < 0 else x)
# # data[data.label.isnull()][['id', 'forecastVolum']].round().astype(int).to_csv(path + '/sub/sub.csv', index=False)
# #
# # ## 简单版本，预测下一个月。
# #
# #
# # #
# # # xgb_model = xgb.XGBRegressor(
# # #     max_depth=5,
# # #     learning_rate=0.01,
# # #     objective='reg:linear',
# # #     eval_metric='mae',
# # #     subsample=0.8, colsample_bytree=0.5,
# # #     n_estimators=3000,
# # #     reg_alpha=0.,
# # #     reg_lambda=0.001,
# # #     min_child_weight=50,
# # #     n_jobs=8,
# # #     seed=42,
# # # )
# # #
# # # data, predict_label = get_predict_w(xgb_model, data, label='temp_label',
# # #                                     feature=features, random_state=2018, n_splits=5)
# # #
# # # data['xgb'] = data[predict_label]
# # #
# # # ctb_params = {
# # #     'n_estimators': 10000,
# # #     'learning_rate': 0.02,
# # #     'random_seed': 4590,
# # #     'reg_lambda': 0.08,
# # #     'subsample': 0.7,
# # #     'bootstrap_type': 'Bernoulli',
# # #     'boosting_type': 'Plain',
# # #     'one_hot_max_size': 10,
# # #     'rsm': 0.5,
# # #     'leaf_estimation_iterations': 5,
# # #     'use_best_model': True,
# # #     'max_depth': 6,
# # #     'verbose': -1,
# # #     'thread_count': 4
# # # }
# # #
# # # ctb_model = ctb.CatBoostRegressor(**ctb_params)
# # #
# # # data, predict_label = get_predict_w(ctb_model, data, label='temp_label',
# # #                                     feature=features,
# # #                                     random_state=2019, n_splits=5, model_type='ctb')
# # #
# # # data['ctb'] = data[predict_label]
# # #
# # # data['t_label'] = data['lgb_mse'] * 0.4 + data['xgb_mse'] * 0.1 + data['ctb_mse'] * 0.4 + data['lgb_mae'] * 0.1